In [9]:
%load_ext autoreload
%autoreload 1

import sys
sys.path.append("../../utils/")

from GraphAPI import GraphCreator

%aimport GraphAPI

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Generating Graph from Entry Point

1. We initialize our GraphCreator class and check how many new nodes we will need to query. 

In [24]:
gc = GraphCreator("Sonata cycle")
print("Number of Links to Search:", len(gc.next_links))
print(gc.intro_nodes)

Number of Links to Search: 147
[]


2. We query all the nodes linked to from the entry point (expand our network one level for each node).

In [25]:
gc.expand_network(group_size=2, timeout=5, log_progress=False)

3. Since some nodes will likely have linked to articles through a redirect link, we need to traverse our graph ensure that all redirects are assigned to the correct nodes. Once all redirects have been dealt with, we remove any old redirect node. 

In [26]:
gc.redraw_redirects()

4. Edges are weighted by how many categories two connected nodes have in common. Once we have all our nodes, and we have dealt with redirects, we can add edge weights for our entire graph. 

In [27]:
gc.update_edge_weights()
gc.get_edge_weights().head()

,source_node,target_node,edge_weight
0,Sonata for two clarinets,Sonata for clarinet and bassoon,5
1,"Six Sonatas for Violin and Harpsichord, BWV 10...",Schubert's compositions for violin and piano,5
2,Sonata for clarinet and bassoon,Violin Sonata (Poulenc),5
3,Cello Sonata (Chopin),Cello Sonata (Rachmaninoff),5
4,Cello Sonata (Poulenc),Violin Sonata (Poulenc),5


# Getting Our Feature Set

There are two options when generating the feature set:

1. we can generate a standard feature set with only the features themselves. To do this, have the `rank` parameter set to `False`.
2. We can generate a ranked feature set (set `rank` equal to `True`). For each parameter, this will rank them in order of _best_ to _worst_ (this could be ascending or descending, depending on the context of the feature).

After running `get_features_df`, the feature set will be saved in the GraphCreator instance as `feature_df`

In [28]:
features_df = gc.get_features_df(rank=False)
gc.features_df

,node,degree,category_matches_with_source,in_edges,out_edges,centrality,dispersion,page_rank,adjusted_reciprocity,shortest_path_length_from_entry,shortest_path_length_to_entry
0,Ludwig van Beethoven,4870,0,4187.0,683.0,4.755957e-02,0.500000,0.119269,902.0,1.0,2.0
1,Music theory,2334,0,1649.0,685.0,5.848027e-03,0.000000,0.034140,300.0,1.0,2.0
2,Index of music articles,2037,0,114.0,1923.0,7.839184e-04,NaN,0.002043,148.0,2.0,1.0
3,Movement (music),1545,0,1497.0,48.0,2.952267e-02,0.000000,0.025023,86.0,1.0,3.0
4,Sonata form,1019,2,711.0,308.0,1.033019e-01,0.506667,0.015225,362.0,2.0,2.0
5,Sonata,959,0,706.0,253.0,1.055225e-01,0.529412,0.011528,228.0,1.0,1.0
6,Variation (music),908,0,567.0,341.0,1.402633e-02,0.166667,0.015096,362.0,1.0,2.0
7,Musical form,874,1,456.0,418.0,1.263758e-02,0.142857,0.012216,226.0,2.0,2.0
8,Motif (music),704,0,590.0,114.0,1.278218e-02,0.181818,0.014736,156.0,1.0,3.0
9,Arcangelo Corelli,603,0,407.0,196.0,9.540679e-02,0.000000,0.007143,210.0,1.0,1.0


## Similarity Rank

Two articles are more similar the more categories they share and the closer they are to each other. 

In [33]:
gc.rank_similarity()
gc.features_df[["node", "category_matches_with_source", "similarity_rank", "adjusted_reciprocity"]].sort_values(["similarity_rank"], ascending=False).head(10)
# gc.features_df[gc.features_df.category_matches_with_source == 1]

0    14114
Name: similarity_rank, dtype: int64

In [ ]:
gc.intro_nodes

# Basic Plotting

In [ ]:
sns.pairplot(features_df)

In [ ]:
sns.heatmap(features_df.corr())

# Intro Node Values

In [ ]:
intro_nodes = gc.intro_nodes
intro_nodes_df = None
for node in intro_nodes:
    if df is None:
        intro_nodes_df = pd.DataFrame(features_df[features_df.node == node])
    else:
        intro_nodes_df = pd.concat([intro_nodes_df, pd.DataFrame(features_df[features_df.node == node])])
    
intro_nodes_df

In [ ]:
sns.pairplot(intro_nodes_df, hue="node")